In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from numpy.ma.core import sort


#initiating lists
watername = []
county =[]
species =[]
quantity =[]
length = []
stockdate = []
images = []
description = []

#initiated 2 counters. x for counting list position for species and i for counter of all loops
x = 0
i = 0

#accessing website 
url = "https://dwrapps.utah.gov/fishstocking/Fish?y=2021"
req = requests.get(url)
soup = BeautifulSoup(req.text, 'lxml')

#each for loop follows the same format as this one. 
#loops through to find the text of all td tags with the specified class name
for td in soup.find_all('td',{'class':'watername'}):
  #appends each td.text to the list and incriments the counter i
  watername.append(td.text) 
  i += 1
  # checks if i is over 500, if it is, i is set back to 0 for the next loop to use and breaks the initial for loop 
  if i > 500:
    i=0
    break

for td in soup.find_all('td',{'class':'county'}):
  county.append(td.text)
  i += 1
  if i > 500:
    i=0
    break

for td in soup.find_all('td',{'class':'species'}):
  species.append(td.text)
  i += 1
  
  #for every species position, go to the respected wikipedia page for that fish and grap the opening paragraph and image. 
  #sets a variable name and begins to format it to wikipedias liking to append to wikipedia's url
  fish_name = td.text.replace(' ','_')
  fish_name = fish_name.lower()

  #checks each species position and depending upon the name, refines it further to how wikipedia likes. 
  if (species[x]== 'RAINBOW'):
    fish_name = "Rainbow_trout"

  elif (species[x]=='CUTTHROAT'):
    fish_name = 'Cutthroat_trout'

  elif (species[x] == 'BASS LARGEMOUTH'):
    fish_name = 'Largemouth_bass'

  elif (species[x] == 'CRAPPIE BLACK'):
    fish_name = "Black_crappie"

  elif (species[x] == 'GRAYLING ARCTIC'):
    fish_name = 'Arctic_grayling'

  elif (species[x] == 'KOKANEE'):
    fish_name = 'Kokanee_salmon'

  elif (species[x] == 'MUSKIE TIGER'):
    fish_name = 'Tiger_muskellunge'

  elif (species[x] == 'SPLAKE'):
    fish_name = 'Splake'

  elif (species[x] == 'SUCKER JUNE'):
    fish_name = "June_sucker"

  elif (species[x] == 'SUNFISH BLUEGILL'):
    fish_name = 'Bluegill'

  elif (species[x] == 'WALLEYE'):
    fish_name = 'Walleye'

  else:
    fish_name ="Hybrid_striped_bass"


  image_url = f"https://en.wikipedia.org/wiki/{fish_name}"
  req2 = requests.get(image_url)
  soup2 = BeautifulSoup(req2.text, 'lxml')
  img = soup2.find('img')['src']
  images.append(img)
  desc = soup2.find_all('p')
  description.append(desc[2].text)
  
  x += 1
  if i > 500:
    i=0
    break


for td in soup.find_all('td',{'class':'quantity'}):
  quantity.append(td.text)
  i += 1
  if i > 500:
    i=0
    break

for td in soup.find_all('td',{'class':'length'}):
  length.append(td.text)
  i += 1
  if i > 500:
    i=0
    break

for td in soup.find_all('td',{'class':'stockdate'}):
  stockdate.append(td.text)
  i += 1
  if i > 500:
    i=0
    break

#creates dataframe

df = pd.DataFrame(columns=['id','watername','county','species','quantity','length','stockdate','images', 'description'])
df.set_index('id', inplace=True)

#adds the lists to the df columns 
df['watername'] = watername
df['county'] = county
df['species'] = species
df['quantity'] = quantity
df['length'] = length
df['stockdate'] = stockdate
df['images'] = images
df['description'] = description

df.sort_values(by=['stockdate'], ascending=False)





,watername,county,species,quantity,length,stockdate,images,description
id,,,,,,,,
369,CURRANT CR RES,WASATCH,TIGER TROUT,1006,6.93,11/29/2021,//upload.wikimedia.org/wikipedia/en/thumb/d/d5...,They became part of aquaculture in the United ...
217,BROWNS DRAW RES,DUCHESNE,TIGER TROUT,499,6.93,11/29/2021,//upload.wikimedia.org/wikipedia/en/thumb/d/d5...,They became part of aquaculture in the United ...
216,BROWNS DRAW RES,DUCHESNE,TIGER TROUT,499,6.93,11/29/2021,//upload.wikimedia.org/wikipedia/en/thumb/d/d5...,They became part of aquaculture in the United ...
74,BARTHOLOMEW PARK POND,UTAH,TIGER TROUT,1500,6.59,11/24/2021,//upload.wikimedia.org/wikipedia/en/thumb/d/d5...,They became part of aquaculture in the United ...
430,DEER CR RES,WASATCH,RAINBOW,1230,10.68,11/22/2021,//upload.wikimedia.org/wikipedia/en/thumb/e/e7...,Adult freshwater stream rainbow trout average ...
...,...,...,...,...,...,...,...,...
296,CLINTON POND,DAVIS,RAINBOW,499,9.44,03/08/2021,//upload.wikimedia.org/wikipedia/en/thumb/e/e7...,Adult freshwater stream rainbow trout average ...
163,BOUNTIFUL POND,DAVIS,RAINBOW,799,9.44,03/01/2021,//upload.wikimedia.org/wikipedia/en/thumb/e/e7...,Adult freshwater stream rainbow trout average ...
228,BURRASTON PONDS,JUAB,BROOK TROUT,201,15.51,02/08/2021,//upload.wikimedia.org/wikipedia/en/thumb/d/d5...,They became part of aquaculture in the United ...


In [4]:
df.to_csv('/content/sample_data/fishStockingData.csv')